In [53]:
# Jayant Jain 
# Run the notebook to check the working.
# Checked working on Win 10
# Code takes care of all the errors mentioned and the corner cases

In [100]:
# Links for reference:
# https://en.wikipedia.org/wiki/Key%E2%80%93value_database
# https://www.w3schools.com/js/js_json_objects.asp
# https://en.wikipedia.org/wiki/Thread_safety

In [48]:
import time
import sys
import threading 
from threading import*



# Server

In [97]:
map={} # global data storage

In [52]:
def create(key,value,timeout=0):  # timeout provided in seconds 
    if key in map:
        print("Error !! Key is already stored") #error mmsg
    else:
        if(key.isalpha()):   # string key      #1073741824 bytes ==1 Gb   
            if sys.getsizeof(map)<(1073741824) and sys.getsizeof(value)<=(16*1024): #Check file size<=1GB and Json obj size<=16kb 
                if timeout==0:
                    l=[value,-1]
                else:
                    l=[value,time.time()+timeout]  #adding timeout incase its not zero
                if len(key)<=32:# key is max of 32 chars
                    map[key]=l 
            else:
                print("Error !! Memory limit")#error mssg
        else:
            print("Error !! Key should have alphabet only")#error mssg


In [50]:
def delete(key):
    if key not in map:
        print("Error !! Key is not in Database") #error mssg
    else:
        list=map[key]
        if list[1]!=-1: # time to live parameter isnt -1(means its provided by user)
            current_time=time.time()
            if current_time<list[1]: #Expiry & current time compared
                del map[key]
                print("Success! key is now deleted")
            else:
                print("Error !! time to live off  expired") #error as time to live has expired so cant delete it
        else:# time to live is -1 then just delete the key
            del map[key]
            print("Success! key is now deleted")

In [49]:
def read(key):
    if key not in map:
        print("Error !! Key is not in Database") #error mssg
    else:
        list=map[key]
        if list[1]!=-1: # time to live parameter isnt -1(means its provided by user)
            current_time=time.time()
            if current_time<list[1]:#Expiry & current time compared
                mapping=str(key)+" : "+str(list[0]) # Key - JSon pair returned from DB
                return mapping
            else:
                print("Error !! time to live off  expired") #error mssg
        else:
            mapping=str(key)+" : "+str(list[0])
            return mapping


# Client

## Testcase 1

In [89]:
json1={  "brand": "Ford",
  "model": "Mustang",
  "year": 1964}

create("car",json1)
#to create a key with key & json obj given and no time-to-live property

json2=[23,12]
create("Money",json2,200) 
#to create a key with key & json obj given and with time-to-live property value given(number of seconds)

print(read("car"))
#PRINTS key in Json object format 'key_name:value'

print(read("Money"))
#PRINTS key in Json object format 'key_name:value' if the (time to live) is not expired else it throws an ERROR !

json3={"32":"google"}
create("car",json3)
#it returns an error since the key_name already present in datastore
 
delete("car")
#it deletes the given key & json obj from datastore

# #Using Multi threading
json4=["New","Year"]
thread1=Thread(target=(create),args=("moker",json4)) #as per the operation
thread1.start()
thread2=Thread(target=(delete),args=("moker",)) #as per the operation
thread2.start()

print("Final datastore",map)

car : {'brand': 'Ford', 'model': 'Mustang', 'year': 1964}
Money : [23, 12]
Error !! Key is already stored
Success! key is now deleted
Success! key is now deleted
Final datastore {'Money': [[23, 12], 1609154201.7596617]}


## Test Case 2

In [98]:
delete("just_key")

json1={  "brand": "Ford",
  "model": "Mustang",
  "year": 1964}

create("cars24",json1)
#Error! as alphanumeric key with key & json obj given and no time-to-live property

json2=[23,12]
create("Money",json2,10) 
#to create a key with key & json obj given and with time-to-live property value given(number of seconds) as just 10 secs

print(read("Money"))
#PRINTS key in Json object format 'key_name:value' if the (time to live) is not expired else it throws an ERROR !

json3={"10":"FreshWork"}
create("TechCos",json3)

 

print("Final datastore",map)

Error !! Key is not in Database
Error !! Key should have alphabet only
Money : [23, 12]
Final datastore {'Money': [[23, 12], 1609154395.5359027], 'TechCos': [{'10': 'FreshWork'}, -1]}


In [99]:
print(read("Money"))
# throws error as run after 10 sec(time to live expired)

Error !! time to live off  expired
None
